# Oscillator frequency error estimator model

A Kalman filter to estimate and phase and frequency error between two oscillators using a constant velocity (constant frequency) model. Sympy is used to compute simplified equations for this specific system, rather than using matrix operations.

In [ ]:
from sympy import *

In [ ]:
dt = Symbol('\Delta t')
A = Matrix([[1, dt], [0, 1]])
C = Matrix([[1, 0]])

var('q_theta q_f r')
Q = Matrix.diag([q_theta, q_f])
R = Matrix.eye(1) * r

I = Matrix.eye(2)
var('theta f')
x = Matrix([[theta], [f]])
var('p00 p01 p10 p11')
P = Matrix([[p00, p01], [p10, p11]])

### Predict

First, propagate forward the state:
$$\mathbf{x}_{k|k-1} = \textbf{A} \mathbf{x}_k$$

In [ ]:
x_kkm1 = A @ x
display(x_kkm1)

Then, propagate the covariance:
$$\mathbf{P}_{k|k-1} = \mathbf{A} \mathbf{P}_{k-1|k-1} \mathbf{A}^\mathsf{T} + \Delta t^2\mathbf{Q}$$

In [ ]:
P_kkm1 = (A @ P @ A.T + Q * dt**2)
display(P_kkm1)

Compute the Kalman gain:
$$\mathbf{K}_k = \mathbf{P}_{k|k-1} \mathbf{C}^\mathsf{T} \left(\mathbf{C} \mathbf{P}_{k|k-1} \mathbf{C}^\mathsf{T} + \mathbf{R}\right)$$

In [ ]:
K = P @ C.T @ (C @ P @ C.T + R).inv()
display(K)

Update state:
$$\mathbf{x}_{k|k} = \mathbf{K}(\mathbf{z} - \mathbf{C} \mathbf{x})$$

In [ ]:
var('theta_m')
K = Matrix([[k0], [k1]])
z = Matrix([[theta_m]])
x_kk = x + K @ (z - C @ x)
display(x_kk)

Update covariance:
$$\mathbf{P}_{k|k} = (\mathbf{I} - \mathbf{K}_k \mathbf{C}) \mathbf{P}_{k|k-1} (\mathbf{I} - \mathbf{K}_k \mathbf{C})^\mathsf{T} + \mathbf{K}_k \mathbf{R} \mathbf{K}_k^\mathsf{T}$$

In [ ]:
P_kk = ((Matrix.eye(2) - K @ C) @ P @ (Matrix.eye(2) - K @ C).T + K @ R @ K.T).applyfunc(simplify)
display(P_kk)

Substitute to simplify:

In [ ]:
P_kk.subs(K[0], k0).subs(K[1], k1)